In [17]:
# Useful links: 
# https://developers.google.com/optimization/mip/mip_example

In [18]:
#!pip install ortools

In [19]:
# utility file used to generate mzn data files from the dat files
import numpy as np
# open the file in Instances folder
f = open("Instances/inst02.dat", "r")
# the first line is the number of couriers
m = int(f.readline())
# the second line is the number of items
n = int(f.readline())
# the third line is the load size of each courier
load_size = [int(x) for x in f.readline().split()]
# the fourth line is the size of each item
item_size = [int(x) for x in f.readline().split()]
# the rest is the distance matrix
distance = []
for i in range(n+1):
    distance.append([int(x) for x in f.readline().split()])
# close the file
f.close()
print("couriers:", m)
print("items:", n)
print("load_size:", load_size)
print("item_size:", item_size)
# output the distance matrix as a numpy array
distance = np.array(distance)
print("distance:\n", distance)

couriers: 6
items: 9
load_size: [190, 185, 185, 190, 195, 185]
item_size: [11, 11, 23, 16, 2, 1, 24, 14, 20]
distance:
 [[  0 199 119  28 179  77 145  61 123  87]
 [199   0  81 206  38 122  55 138  76 113]
 [119  81   0 126  69 121  26 117  91  32]
 [ 28 206 126   0 186  84 152  68 130  94]
 [169  38  79 176   0  92  58 108  46  98]
 [ 77 122 121  84 102   0 100  16  46  96]
 [145  55  26 152  58 100   0  91  70  58]
 [ 61 138 113  68 118  16  91   0  62  87]
 [123  76  91 130  56  46  70  62   0  66]
 [ 87 113  32  94  94  96  58  87  66   0]]


In [20]:
from ortools.linear_solver import pywraplp

def solve_courier_problem(courier_capacities, item_sizes, item_distances):
    num_couriers = len(courier_capacities)
    num_items = len(item_sizes)

    solver = pywraplp.Solver.CreateSolver('SCIP')
    max_distance = solver.IntVar(0, solver.infinity(), 'max_distance')

    # Create variables
    assignment = [[[solver.IntVar(0, 1, f'assignment_{i}_{j}_{k}') for j in range(num_items + 1)] for k in range(num_items + 1)] for i in range(num_couriers)]
    
    # The diagonal of the matrix is 0
    solver.Add(sum(assignment[i][j][j] for i in range(num_couriers) for j in range(num_items + 1)) == 0)
    
    # Create constraints: items are assigned to at most one courier
    for j in range(num_items):
        solver.Add(sum(assignment[i][j][k] for i in range(num_couriers) for k in range(num_items + 1)) == 1)
        solver.Add(sum(assignment[i][k][j] for i in range(num_couriers) for k in range(num_items + 1)) == 1)
    
    for i in range(num_couriers):
        # We set boundaries for max distance, so that we can minimize it later
        solver.Add(sum(assignment[i][j][k] * item_distances[j][k] for j in range(num_items + 1) for k in range(num_items + 1)) <= max_distance)
        # Create constraints: each courier carries at least one item
        solver.Add(sum(assignment[i][num_items][k] for k in range(num_items + 1)) == 1)
        # Create constraint: courier capacities are respected
        solver.Add(sum(assignment[i][j][k] * item_sizes[j] for j in range(num_items) for k in range(num_items + 1)) <= courier_capacities[i])
        for j in range(num_items + 1):
            # Create constraint: n arcs in, n arcs out
            solver.Add(sum(assignment[i][j][k] for k in range(num_items + 1)) == sum(assignment[i][k][j] for k in range(num_items + 1)))
    
    # Create constraint: Eliminate subtours using the Miller-Tucker-Zemlin (MTZ) formulation
    u = [[solver.IntVar(0, solver.infinity(), f'u_{i}_{j}') for j in range(num_items + 1)] for i in range(num_couriers)]

    for i in range(num_couriers):
        for j in range(0, num_items + 1):
            for k in range(0, num_items + 1):
                if j != k and j != num_items:
                    # I add the constraint of the MTZ formulation
                    solver.Add(u[i][j] - u[i][k] + 1 <= (num_items - 1) * (1 - assignment[i][j][k]))
    
    solver.Minimize(max_distance)
    
    solver.set_time_limit(300000)
    
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        for i in range(num_couriers):
            print(f'Courier {i} takes item: ')
            print('{')
            for j in range(num_items):
                for k in range(num_items + 1):
                    if assignment[i][j][k].solution_value() > 0:
                        print(f' -> {j}')
            print('}')
        print('Max distance:', max_distance.solution_value())
        print('Objective value:', solver.Objective().Value())
        # I print the values of assignment
        # for each courier i create and print a item matrix
        print()
        for i in range(num_couriers):
            print(f'Courier {i} item matrix:')
            for j in range(num_items + 1):
                for k in range(num_items + 1):
                    print(int(assignment[i][j][k].solution_value()), end=' ')
                print()
            print()
        # I print the values of u
        print()
        print('u matrix:')
        for i in range(num_couriers):
            for j in range(num_items + 1):
                print(int(u[i][j].solution_value()), end=' ')
            print()
        print(item_distances)
    else:
        print('The problem does not have an optimal solution.')

solve_courier_problem(load_size, item_size, distance)


Solution:
Courier 0 takes item: 
{
 -> 7
 -> 8
}
Courier 1 takes item: 
{
 -> 5
}
Courier 2 takes item: 
{
 -> 4
}
Courier 3 takes item: 
{
 -> 1
 -> 6
}
Courier 4 takes item: 
{
 -> 0
 -> 3
}
Courier 5 takes item: 
{
 -> 2
}
Max distance: 226.00000000000003
Objective value: 226.00000000000003

Courier 0 item matrix:
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 
0 0 0 0 0 0 0 1 0 0 
0 0 0 0 0 0 0 0 1 0 

Courier 1 item matrix:
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 1 0 0 0 0 

Courier 2 item matrix:
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 1 0 0 0 0 0 

Co